In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import *
from sklearn.preprocessing import *
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import *
import tensorflow as tf
from sklearn.metrics import *
import joblib

In [34]:
df_1 = pd.read_csv("train1.csv")
df_2 = pd.read_csv("train2.csv")
df_3 = pd.read_csv("train3.csv")

df = pd.concat([df_1, df_2, df_3],axis=1)

df.head()

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,...,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,datasetId,condition
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,11.001565,...,15.522602,0.421047,1.514737,3686.666157,65.018055,0.015380,2.139754,1.163485,2,0
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,9.170129,...,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,2.174499,1.084711,2,1
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,11.533417,...,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,2.135350,1.176315,2,1
3,881.757864,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,11.119476,...,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,2.178341,1.179688,2,0
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,13.590641,...,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,2.221121,1.249612,2,0


In [35]:
df_X = df.drop(columns=["condition"])
df_y = df["condition"]

df_X.columns

Index(['MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD', 'HR',
       'pNN25', 'pNN50', 'SD1', 'SD2', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR',
       'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'VLF', 'VLF_PCT',
       'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF',
       'HF_LF', 'sampen', 'higuci', 'datasetId'],
      dtype='object')

In [36]:
important_features = ["MEAN_RR", "RMSSD", "pNN25", "pNN50", "LF", "HF", "LF_HF"]
df_X_1 = df_X[important_features]
df_y_1 = df_y.copy()

In [37]:
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(df_X_1),columns=df_X_1.columns)
y = df_y_1.copy()

In [38]:
y.unique()

array([0, 1, 2], dtype=int64)

### Testing the model

In [39]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=np.random.randint(0,1000))
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=np.random.randint(0,1000))
X_train.shape

(236344, 7)

In [40]:
model_1 = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Dense(units=64, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=32, activation="relu"),
    tf.keras.layers.Dense(units=16, activation="relu"),
    tf.keras.layers.Dense(units=8, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="softmax")
])

In [41]:
model_1.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [42]:
model_1.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, callbacks=tf.keras.callbacks.EarlyStopping(patience=2), validation_data=(X_val, y_val), shuffle=True)

Epoch 1/100
7386/7386 [==============================] - 13s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2854 - val_loss: 0.0000e+00 - val_accuracy: 0.2847
Epoch 2/100
7386/7386 [==============================] - 12s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2854 - val_loss: 0.0000e+00 - val_accuracy: 0.2847
Epoch 3/100
7386/7386 [==============================] - 15s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2854 - val_loss: 0.0000e+00 - val_accuracy: 0.2847


In [43]:
cat_cross_entropy_loss, accuracy_score = model_1.evaluate(X_test, y_test)

2309/2309 [==============================] - 3s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2826


In [44]:
cat_cross_entropy_loss

0.0

In [45]:
round(accuracy_score,2)

0.28

In [46]:
y_pred = model_1.predict(X_test)

2309/2309 [==============================] - 3s 1ms/step


In [47]:
root_mean_squared_error = np.round(np.sqrt(mean_squared_error(y_test, y_pred)),2)
root_mean_squared_error

0.85

### Final Prediction of testing set

In [48]:
X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(X,y,test_size=0.2,random_state=np.random.randint(0,1000))
test_df = pd.read_csv("test.csv")
X_test_2 = pd.DataFrame(sc.fit_transform(test_df[important_features]),columns=important_features)
y_test_2 = test_df["condition"]

In [49]:
model_2 = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Dense(units=64, activation="relu", input_shape=(X_train_2.shape[1],)),
    tf.keras.layers.Dense(units=32, activation="relu"),
    tf.keras.layers.Dense(units=16, activation="relu"),
    tf.keras.layers.Dense(units=8, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="softmax")
])

In [50]:
model_2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [51]:
model_2.fit(X_train_2, y_train_2, batch_size=32, epochs=100, verbose=1, callbacks=tf.keras.callbacks.EarlyStopping(patience=2), validation_data=(X_val_2, y_val_2), shuffle=True)

Epoch 1/100
9233/9233 [==============================] - 16s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2848 - val_loss: 0.0000e+00 - val_accuracy: 0.2844
Epoch 2/100
9233/9233 [==============================] - 15s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2848 - val_loss: 0.0000e+00 - val_accuracy: 0.2844
Epoch 3/100
9233/9233 [==============================] - 15s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2848 - val_loss: 0.0000e+00 - val_accuracy: 0.2844


In [52]:
cat_cross_entropy_loss_2, accuracy_score_2 = model_2.evaluate(X_test_2, y_test_2)

1283/1283 [==============================] - 2s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2871


In [53]:
cat_cross_entropy_loss_2

0.0

In [54]:
round(accuracy_score_2,2)

0.29

In [55]:
y_pred_2 = model_2.predict(X_test_2)

1283/1283 [==============================] - 1s 939us/step


In [56]:
root_mean_squared_error_2 = np.round(np.sqrt(mean_squared_error(y_test_2, y_pred_2)),2)
root_mean_squared_error_2

0.84

In [57]:
joblib.dump(value=sc,filename="scaler.joblib",compress=9)
model_2.save("model.h5")